# Opioid Data - Standardize Timeseries
HW #2 Part 2 - Timeseries.  
Use all rows per patient from about 30 consecutive days.
Standardize all of it.

## Preprocessing
Input characteristics: 
Patient files are in one of two directories: R or NR.
Each patient is represented by one CSV file.
Each row of each CSV contains readings from one day.
Days are in order and mostly sequential (at least one day is missing in one patient).
Most patients have 30 days but some have less.

Loaded data structure characteristics:
Labels columns for cohort (R or NR), patient (2005_S3), date.
Labels rows are num_cohorts * num_patients * num_dates.
Features columns are measurements without dates.
Features rows are same as Labels rows.

We will assume timeseries records are equally spaced at one per day.
This is mostly true with a few abberations.
For example, R patient 2060_S3 is missing the record between 2020-06-01 and 2020-06-03.

We will align all patient records by measuring days-since-start.
We will ignore the specific dates which can be in different months for different patients.
Thus, we are assuming month of year has no effect on the response variable R/NR.

In [1]:
from os import listdir
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import time

In [2]:
pathR='data/ChunkedData_R/'
pathN='data/chunkedData_NR/'
CLASS_SEPARATOR=13  # data[:13] vs data[13:]
WITH_VARIANCE_COLUMNS=True   # Use mean and variance per patient

In [3]:
COL_LIST = ['cohort','patient_name','date']
labels_list = []
features_df = pd.DataFrame()
# Read one CSV file. 
# Load global lists
def load_patient (filepath,cohort,patient_name):
    global labels_df
    global features_df
    one_patient = pd.read_csv(filepath)
    rows,cols = one_patient.shape
    features_df = features_df.append(one_patient)
    for rec in range(0,rows):
        one_label=(cohort,patient_name,one_patient.loc[rec]['Date'])
        labels_list.append(one_label)

In [4]:
# Read directory of CSV files (R or NR). 
# Given directory, load all the patients in that directory.
# We use filenames as patient names.
def load_cohort (cohort,directory):
    file_names = listdir(directory)
    for fp in file_names:
        dfp = directory+fp
        one_name = fp.split('.')[0]  # strip away .csv suffix
        one_name = one_name[6:]    # strip away Daily_ prefix
        one_patient = load_patient(dfp,cohort,one_name)

In [5]:
print(len(listdir(pathR))," files of type R")
load_cohort('R',pathR)
print(len(listdir(pathN))," files of type N")
load_cohort('N',pathN)
features_df = features_df.drop('Date',axis=1) 
features_df.head()

14  files of type R
26  files of type N


,Morning_Question1,Morning_Question2,Morning_Question3,Morning_Question4,Morning_Question5,Morning_Question6,Afternoon_Question1,Afternoon_Question2,Afternoon_Question3,Afternoon_Question4,...,HR_mean,HR_var,HR_std,HR_sk,HR_ku,Stress_mean,Stress_var,Stress_std,Stress_sk,Stress_ku
0,2,2,4,4,3,4,4,4,4,4,...,74.290849,320.934322,17.914640,1.071010,0.770627,39.173780,769.416191,27.738352,0.451429,-1.319960
1,1,2,4,4,2,4,2,5,5,4,...,74.401459,308.214628,17.556042,1.327296,1.331250,38.691558,485.933870,22.043908,0.706097,-0.149387
2,2,1,1,4,4,5,4,4,4,4,...,74.329967,292.800716,17.111421,0.993076,0.316197,35.116129,880.102975,29.666530,0.565035,-1.181336
3,2,2,3,4,3,4,2,2,3,4,...,77.153765,166.815153,12.915694,1.120305,3.849928,52.852547,349.389489,18.691963,-0.040921,-0.137957
4,2,2,4,4,3,5,4,4,4,4,...,80.128234,197.631934,14.058163,1.264925,0.945865,45.834320,377.319680,19.424718,0.539156,-0.363470


In [6]:
# Labels is a list of triples: cohort, patient name, test date.
# Labels is just identifying info; no data.
# Labels and Features are in one-to-one correspondence.
print("labels:",len(labels_list)," like ",labels_list[0])
# Features is a dataframe with named columns.
# Features is just features; no identifying info.
print("features:",features_df.shape)

labels: 1004  like  ('R', '2060_S3', '2020-05-21')
features: (1004, 259)


## Scaling and covariance
Normalize by subtracting the column mean from every column value.  
Since columns have widely different numerical ranges,   
also normalize by making each column have unit variance.  
Note: without normalization, the covariance plot would be all black except for the few features with large absolute values.

In [7]:
# Standardize features by shifting the mean to zero and scaling to unit variance.
# Subtract the mean and divide by the std.dev: z = (x - u) / s
def scale_features(X):
    s = StandardScaler()
    z = s.fit_transform(X)
    return z
scaledMeans = scale_features(meansAll)
column_names = meansAll.columns

NameError: name 'meansAll' is not defined

In [ ]:
print("Column Names:")
print(column_names)
print("Scaled Means & Variances:")
scaledMeans

In [ ]:
pathR = None
pathN = None
filesR = None
filesN = None
meansR = None
meansN = None
meansAll = None